# Webscraping Traditional NBA Pipeline

In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as bs
from time import sleep
import requests
import pandas as pd
import numpy as np

In [2]:
# Make a url list
url_base = 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season='
url_list = []
a = 2021

while a >= 1996:
    b = a+1
    new_url = url_base + str(a) + "-" + str(b)[-2:] + "&SeasonType=Regular%20Season"
    a -=1
    url_list.append(new_url)

In [3]:
url_list[:5]

['https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2021-22&SeasonType=Regular%20Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2020-21&SeasonType=Regular%20Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2019-20&SeasonType=Regular%20Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2018-19&SeasonType=Regular%20Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2017-18&SeasonType=Regular%20Season']

In [7]:
dataframe_list = []
n=1 # Make a counter to see how many webpages are scraped

# Scrape stat table for each url in url_list
for url in url_list:
    # provide path to the Edge driver
    s=Service("C:\\Users\sandr\OneDrive\Desktop\Drivers\msedgedriver.exe")
    driver = webdriver.Edge(service=s)
    
    # Make python pause for some time
    sleep(2)

    # Get the page HTML
    try:
        driver.get(url)
        # Make python pause for some time
        sleep(3)
    except WebDriverException:
        print("Webdriver is acting immediately after ‘closing’ the browser")

    # Click on accept cookies button
    driver.find_element(By.XPATH, "/html/body/div[5]/div[2]/div/div[1]/div/div[2]/div/button").click()

    # Make python pause for some time
    sleep(4)

    # Select table page
    select_page = Select(driver.find_element(By.XPATH,"/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select"))
    select_page.select_by_visible_text('All')

    # Use BeautifulSoup and lxml extension to parse, save in variable nba_soup
    nba_soup = bs(driver.page_source, 'lxml')
    
    # Locate the table on the page
    table = nba_soup.find('table')
   
    # Find all the headers in the columns
    headers=[]
    for i in table.find_all('th'):
        title = i.text.replace("\n", "").strip()
        headers.append(title)
    
    # Remove redundant headers
    headers_2 = ['PLAYER','TEAM','AGE','GP','W','L','MIN','PTS','FGM','FGA',
                 'FG%','3PM','3PA','3P%','FTM','FTA','FT%','OREB','DREB', 
                 'REB','AST','TOV','STL','BLK','PF','FP','DD2','TD3','+/-']

    # Create a dataframe using the headers as column names
    nba_stat = pd.DataFrame(columns = headers_2) 

    # Create a for loop to fill nba_stat
    for i in table.find_all('tr')[1:]:
        row_data = i.find_all('td')[1:]
        row =[j.text.replace("\n", "") for j in row_data] 
        length = len(nba_stat)
        nba_stat.loc[length] = row
    
    print("Completed:", n, "webpage(s) scraped.")
    n +=1 # add 1 to counter every successful page retrieved
        
    # Close all browsers
    driver.quit()

    # Append table to list of dataframes
    dataframe_list.append(nba_stat)

Webdriver is acting immediately after ‘closing’ the browser
Completed: 1 webpage(s) scraped.
Completed: 2 webpage(s) scraped.
Webdriver is acting immediately after ‘closing’ the browser
Completed: 3 webpage(s) scraped.
Webdriver is acting immediately after ‘closing’ the browser
Completed: 4 webpage(s) scraped.
Webdriver is acting immediately after ‘closing’ the browser
Completed: 5 webpage(s) scraped.
Completed: 6 webpage(s) scraped.
Completed: 7 webpage(s) scraped.
Completed: 8 webpage(s) scraped.
Webdriver is acting immediately after ‘closing’ the browser
Completed: 9 webpage(s) scraped.
Completed: 10 webpage(s) scraped.
Webdriver is acting immediately after ‘closing’ the browser
Completed: 11 webpage(s) scraped.
Webdriver is acting immediately after ‘closing’ the browser
Completed: 12 webpage(s) scraped.
Webdriver is acting immediately after ‘closing’ the browser
Completed: 13 webpage(s) scraped.
Webdriver is acting immediately after ‘closing’ the browser
Completed: 14 webpage(s) sc

In [8]:
# dataframe_list is a list of all dataframes
dataframe_list

[             PLAYER TEAM AGE  GP   W   L   MIN   PTS              FGM  \
 0        Trae Young  ATL  23  76  40  36  2652  2155              711   
 1     DeMar DeRozan  CHI  32  76  43  33  2743  2118              774   
 2       Joel Embiid  PHI  28  68  45  23  2296  2079              666   
 3      Jayson Tatum  BOS  24  76  49  27  2731  2046              708   
 4      Nikola Jokic  DEN  27  74  46  28  2476  2004              764   
 ..              ...  ...  ..  ..  ..  ..   ...   ...              ...   
 600     Nate Hinton  IND  23   2   0   2     2     0                0   
 601      Sam Dekker  TOR  28   1   1   0     1     0                0   
 602  Trayvon Palmer  DET  27   1   0   1    17     0                0   
 603      Tyler Hall  NYK  25   1   1   0     2     0                0   
 604  Zylan Cheatham  UTA  26   1   0   1     5     0                0   
 
                   FGA  ...               REB              AST  \
 0                1544  ...               28

In [10]:
# Concat all the dataframes into one table
nba_stats = pd.concat(dataframe_list, ignore_index=True)
nba_stats

,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
0,Trae Young,ATL,23,76,40,36,2652,2155,711,1544,...,284,737,303,72,7,128,3535.3,42,0,159
1,DeMar DeRozan,CHI,32,76,43,33,2743,2118,774,1535,...,392,374,181,68,24,178,3244.4,6,0,77
2,Joel Embiid,PHI,28,68,45,23,2296,2079,666,1334,...,796,284,214,77,99,181,3774.2,46,2,368
3,Jayson Tatum,BOS,24,76,49,27,2731,2046,708,1564,...,609,334,217,75,49,174,3432.8,22,0,667
4,Nikola Jokic,DEN,27,74,46,28,2476,2004,764,1311,...,1019,584,281,109,63,191,4337.8,66,19,444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12302,Anthony Miller,ATL,25,1,0,1,14,0,0,5,...,7,0,0,0,0,2,8.4,0,0,-14
12303,Bruce Bowen,MIA,26,1,1,0,1,0,0,0,...,0,0,0,0,1,0,3.0,0,0,3
12304,Cuonzo Martin,MIL,25,3,0,3,13,0,0,7,...,1,1,1,0,0,1,1.7,0,0,-2
12305,Derrick Alston,ATL,24,2,1,1,11,0,0,5,...,4,0,0,0,0,0,4.8,0,0,-7


# Prepare and Clean Data

In [11]:
# Check datatypes
nba_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12307 entries, 0 to 12306
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   PLAYER  12307 non-null  object
 1   TEAM    12307 non-null  object
 2   AGE     12307 non-null  object
 3   GP      12307 non-null  object
 4   W       12307 non-null  object
 5   L       12307 non-null  object
 6   MIN     12307 non-null  object
 7   PTS     12307 non-null  object
 8   FGM     12307 non-null  object
 9   FGA     12307 non-null  object
 10  FG%     12307 non-null  object
 11  3PM     12307 non-null  object
 12  3PA     12307 non-null  object
 13  3P%     12307 non-null  object
 14  FTM     12307 non-null  object
 15  FTA     12307 non-null  object
 16  FT%     12307 non-null  object
 17  OREB    12307 non-null  object
 18  DREB    12307 non-null  object
 19  REB     12307 non-null  object
 20  AST     12307 non-null  object
 21  TOV     12307 non-null  object
 22  STL     12307 non-null

In [12]:
# Get column names
nba_stats.columns

Index(['PLAYER', 'TEAM', 'AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 'FGM', 'FGA',
       'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB',
       'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2', 'TD3', '+/-'],
      dtype='object')

In [13]:
# Change datatypes to numeric for appropriate columns
nba_stats[['AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 'FGM', 'FGA',
                 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 
                 'OREB', 'DREB', 'REB','AST', 'TOV', 'STL', 'BLK', 
                 'PF', 'FP', 'DD2', 'TD3', '+/-']] = nba_stats[['AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 
                                                                      'FGM', 'FGA','FG%', '3PM', '3PA', '3P%', 
                                                                      'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB',
                                                                      'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2', 
                                                                      'TD3', '+/-']].apply(pd.to_numeric)

In [14]:
# Verify datatype changes
nba_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12307 entries, 0 to 12306
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PLAYER  12307 non-null  object 
 1   TEAM    12307 non-null  object 
 2   AGE     12307 non-null  int64  
 3   GP      12307 non-null  int64  
 4   W       12307 non-null  int64  
 5   L       12307 non-null  int64  
 6   MIN     12307 non-null  int64  
 7   PTS     12307 non-null  int64  
 8   FGM     12307 non-null  int64  
 9   FGA     12307 non-null  int64  
 10  FG%     12307 non-null  float64
 11  3PM     12307 non-null  int64  
 12  3PA     12307 non-null  int64  
 13  3P%     12307 non-null  float64
 14  FTM     12307 non-null  int64  
 15  FTA     12307 non-null  int64  
 16  FT%     12307 non-null  float64
 17  OREB    12307 non-null  int64  
 18  DREB    12307 non-null  int64  
 19  REB     12307 non-null  int64  
 20  AST     12307 non-null  int64  
 21  TOV     12307 non-null  int64  
 22

In [17]:
# Look at dimension
nba_stats.shape

(12307, 29)

In [21]:
# Find duplicate rows, if any
duplicateRows = nba_stats[nba_stats.duplicated()]
duplicateRows

,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-


There are no duplicate rows

In [22]:
# Remove columns that are redundant, unnecessary
nba_stats2 = nba_stats.drop(['TEAM','PF','FP',
                          'DD2','TD3','+/-'], axis=1)

nba_stats2.head()

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK
0,Trae Young,23,76,40,36,2652,2155,711,1544,46.0,...,500,553,90.4,50,234,284,737,303,72,7
1,DeMar DeRozan,32,76,43,33,2743,2118,774,1535,50.4,...,520,593,87.7,56,336,392,374,181,68,24
2,Joel Embiid,28,68,45,23,2296,2079,666,1334,49.9,...,654,803,81.4,146,650,796,284,214,77,99
3,Jayson Tatum,24,76,49,27,2731,2046,708,1564,45.3,...,400,469,85.3,85,524,609,334,217,75,49
4,Nikola Jokic,27,74,46,28,2476,2004,764,1311,58.3,...,379,468,81.0,206,813,1019,584,281,109,63


In [23]:
# Look at dimension
nba_stats2.shape

(12307, 23)

### Save file

In [24]:
# Save as csv file
nba_stats2.to_csv('nba_stats.csv', encoding='utf-8', index=False)

In [25]:
# Check that it's saved
nba_stats2 = pd.read_csv('nba_stats.csv', encoding='utf-8')
nba_stats2

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK
0,Trae Young,23,76,40,36,2652,2155,711,1544,46.0,...,500,553,90.4,50,234,284,737,303,72,7
1,DeMar DeRozan,32,76,43,33,2743,2118,774,1535,50.4,...,520,593,87.7,56,336,392,374,181,68,24
2,Joel Embiid,28,68,45,23,2296,2079,666,1334,49.9,...,654,803,81.4,146,650,796,284,214,77,99
3,Jayson Tatum,24,76,49,27,2731,2046,708,1564,45.3,...,400,469,85.3,85,524,609,334,217,75,49
4,Nikola Jokic,27,74,46,28,2476,2004,764,1311,58.3,...,379,468,81.0,206,813,1019,584,281,109,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12302,Anthony Miller,25,1,0,1,14,0,0,5,0.0,...,0,0,0.0,2,5,7,0,0,0,0
12303,Bruce Bowen,26,1,1,0,1,0,0,0,0.0,...,0,0,0.0,0,0,0,0,0,0,1
12304,Cuonzo Martin,25,3,0,3,13,0,0,7,0.0,...,0,0,0.0,1,0,1,1,1,0,0
12305,Derrick Alston,24,2,1,1,11,0,0,5,0.0,...,0,2,0.0,3,1,4,0,0,0,0


In [90]:
test = nba_stats2[nba_stats2['PLAYER'] == 'Stephen Curry']
test

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK
13,Stephen Curry,34,64,45,19,2211,1630,535,1224,43.7,...,275,298,92.3,34,301,335,404,206,85,23
605,Stephen Curry,33,63,37,26,2152,2015,658,1365,48.2,...,362,395,91.6,29,316,345,363,213,77,8
1529,Stephen Curry,32,5,1,4,139,104,33,82,40.2,...,26,26,100.0,4,22,26,33,16,5,2
1681,Stephen Curry,31,69,52,17,2331,1881,632,1340,47.2,...,263,287,91.6,45,324,369,361,192,92,25
2233,Stephen Curry,30,51,41,10,1631,1346,428,864,49.5,...,278,302,92.1,36,225,261,310,153,80,8
2751,Stephen Curry,29,79,65,14,2639,1999,675,1443,46.8,...,325,362,89.8,61,292,353,524,239,142,17
3231,Stephen Curry,28,79,71,8,2700,2375,805,1598,50.4,...,363,400,90.8,68,362,430,527,262,169,15
3707,Stephen Curry,27,80,67,13,2613,1900,653,1341,48.7,...,308,337,91.4,56,285,341,619,249,163,16
4203,Stephen Curry,26,78,50,28,2846,1873,652,1383,47.1,...,308,348,88.5,46,288,334,666,294,128,14
4686,Stephen Curry,25,78,45,33,2983,1786,626,1388,45.1,...,262,291,90.0,59,255,314,539,240,126,12


This tells us that some players play multiple years and will have multiple rows of stats.

It is simplest to compress them into one row per player and get the average for the stats.

Since most rookie contracts are up to 4 years, we'll only look at non-rookie players, i.e. players who have played for 5 or more years.

In [91]:
# Let's see the dataframe of players and the number of years they've played
years_played = nba_stats2['PLAYER'].value_counts()
years_played

Vince Carter      22
Dirk Nowitzki     21
Kobe Bryant       20
Kevin Garnett     20
Jamal Crawford    20
                  ..
Kevin Murphy       1
Kris Joseph        1
Fab Melo           1
Josh Akognon       1
Elmore Spencer     1
Name: PLAYER, Length: 2454, dtype: int64

In [92]:
# Create a dataframe of players and # of years, rename index and columns
years_played = nba_stats2['PLAYER'].value_counts().rename_axis('Players').reset_index(name='counts')
years_played

,Players,counts
0,Vince Carter,22
1,Dirk Nowitzki,21
2,Kobe Bryant,20
3,Kevin Garnett,20
4,Jamal Crawford,20
...,...,...
2449,Kevin Murphy,1
2450,Kris Joseph,1
2451,Fab Melo,1
2452,Josh Akognon,1


In [93]:
years_played.columns

Index(['Players', 'counts'], dtype='object')

In [94]:
# Filter out the rookies, keep players who have played for more than 4 years
years_played2 = years_played[years_played['counts'] >4]
years_played2

,Players,counts
0,Vince Carter,22
1,Dirk Nowitzki,21
2,Kobe Bryant,20
3,Kevin Garnett,20
4,Jamal Crawford,20
...,...,...
1008,Markelle Fultz,5
1009,David Wingate,5
1010,Henry Ellenson,5
1011,Sam Mack,5


In [95]:
# Make a list with the players who have played for more than 4 years
inclusion = years_played2['Players']
inclusion

0         Vince Carter
1        Dirk Nowitzki
2          Kobe Bryant
3        Kevin Garnett
4       Jamal Crawford
             ...      
1008    Markelle Fultz
1009     David Wingate
1010    Henry Ellenson
1011          Sam Mack
1012    Duane Causwell
Name: Players, Length: 1013, dtype: object

In [96]:
nba_nr = nba_stats2[nba_stats2['PLAYER'].isin(inclusion)]
nba_nr

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK
1,DeMar DeRozan,32,76,43,33,2743,2118,774,1535,50.4,...,520,593,87.7,56,336,392,374,181,68,24
2,Joel Embiid,28,68,45,23,2296,2079,666,1334,49.9,...,654,803,81.4,146,650,796,284,214,77,99
3,Jayson Tatum,24,76,49,27,2731,2046,708,1564,45.3,...,400,469,85.3,85,524,609,334,217,75,49
4,Nikola Jokic,27,74,46,28,2476,2004,764,1311,58.3,...,379,468,81.0,206,813,1019,584,281,109,63
5,Giannis Antetokounmpo,27,67,45,22,2204,2002,689,1245,55.3,...,553,766,72.2,134,644,778,388,219,72,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12251,Ben Wallace,22,34,21,13,194,38,16,46,34.8,...,6,20,30.0,25,33,58,2,18,8,11
12275,Antonio Harvey,26,6,6,0,27,15,5,11,45.5,...,5,6,83.3,2,8,10,1,1,0,4
12282,Moochie Norris,23,8,2,6,88,12,4,22,18.2,...,2,5,40.0,3,9,12,23,5,4,0
12302,Anthony Miller,25,1,0,1,14,0,0,5,0.0,...,0,0,0.0,2,5,7,0,0,0,0


In [97]:
# Groupby player to compress each player down to one row, then take the mean of the stats
nba_nr2 = nba_nr.groupby('PLAYER').mean().round(3)
nba_nr2 = nba_nr2.reset_index() # reset the index
nba_nr2

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK
0,A.C. Green,35.000,75.800,35.800,40.000,1881.000,444.600,180.200,397.000,44.980,...,83.800,123.400,67.000,158.000,312.200,470.200,67.200,51.800,51.800,15.400
1,AJ Price,25.500,43.500,19.833,23.667,654.667,253.833,91.500,240.667,38.000,...,34.500,46.500,62.067,11.333,51.500,62.833,94.500,38.167,20.833,1.500
2,Aaron Brooks,28.100,64.500,35.900,28.600,1343.900,625.900,226.100,547.800,41.090,...,91.700,109.500,82.000,25.400,82.500,107.900,192.500,100.000,35.900,8.700
3,Aaron Gordon,22.500,66.000,29.875,36.125,1909.500,860.875,325.625,707.875,45.962,...,136.250,194.000,70.062,105.125,308.625,413.750,164.125,99.000,48.625,41.875
4,Aaron Gray,26.000,45.429,17.714,27.714,550.000,152.286,62.857,123.429,50.857,...,26.571,47.286,56.586,61.571,108.429,170.000,29.571,37.857,13.429,12.429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,Zach Randolph,28.000,65.647,33.000,32.647,2036.647,1092.824,437.412,927.882,47.041,...,206.294,270.118,75.318,192.824,407.588,600.412,120.529,134.706,49.059,17.706
1009,Zaza Pachulia,27.500,68.625,34.062,34.562,1389.562,463.375,163.000,347.438,46.912,...,137.375,182.938,74.975,147.625,247.062,394.688,89.562,90.250,48.000,23.625
1010,Zeljko Rebraca,32.000,43.000,24.000,19.000,655.600,255.200,97.600,185.200,52.180,...,60.000,75.800,78.900,39.200,98.400,137.600,17.600,40.600,11.400,32.000
1011,Zendon Hamilton,28.500,22.333,8.500,13.833,259.333,97.500,29.333,66.667,40.383,...,38.833,58.833,70.767,30.667,46.667,77.333,5.500,18.000,6.500,4.667


In [98]:
# Verify each player only has one row
test2 = nba_nr2[nba_nr2['PLAYER'] == 'Stephen Curry']
test2

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK
893,Stephen Curry,28.0,63.538,41.923,21.615,2181.692,1543.385,528.846,1116.923,46.931,...,245.923,270.769,90.838,42.538,252.692,295.231,414.462,199.846,105.154,14.385


## Save into csv file

In [99]:
nba_nr2.to_csv('nba_stats_cleaned.csv', index=False)

In [100]:
# Check that it's saved
nba_stats = pd.read_csv('nba_stats_cleaned.csv')
nba_stats

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK
0,A.C. Green,35.000,75.800,35.800,40.000,1881.000,444.600,180.200,397.000,44.980,...,83.800,123.400,67.000,158.000,312.200,470.200,67.200,51.800,51.800,15.400
1,AJ Price,25.500,43.500,19.833,23.667,654.667,253.833,91.500,240.667,38.000,...,34.500,46.500,62.067,11.333,51.500,62.833,94.500,38.167,20.833,1.500
2,Aaron Brooks,28.100,64.500,35.900,28.600,1343.900,625.900,226.100,547.800,41.090,...,91.700,109.500,82.000,25.400,82.500,107.900,192.500,100.000,35.900,8.700
3,Aaron Gordon,22.500,66.000,29.875,36.125,1909.500,860.875,325.625,707.875,45.962,...,136.250,194.000,70.062,105.125,308.625,413.750,164.125,99.000,48.625,41.875
4,Aaron Gray,26.000,45.429,17.714,27.714,550.000,152.286,62.857,123.429,50.857,...,26.571,47.286,56.586,61.571,108.429,170.000,29.571,37.857,13.429,12.429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,Zach Randolph,28.000,65.647,33.000,32.647,2036.647,1092.824,437.412,927.882,47.041,...,206.294,270.118,75.318,192.824,407.588,600.412,120.529,134.706,49.059,17.706
1009,Zaza Pachulia,27.500,68.625,34.062,34.562,1389.562,463.375,163.000,347.438,46.912,...,137.375,182.938,74.975,147.625,247.062,394.688,89.562,90.250,48.000,23.625
1010,Zeljko Rebraca,32.000,43.000,24.000,19.000,655.600,255.200,97.600,185.200,52.180,...,60.000,75.800,78.900,39.200,98.400,137.600,17.600,40.600,11.400,32.000
1011,Zendon Hamilton,28.500,22.333,8.500,13.833,259.333,97.500,29.333,66.667,40.383,...,38.833,58.833,70.767,30.667,46.667,77.333,5.500,18.000,6.500,4.667
